# Calculating gradients with `SQcircuit`

Using SQcircuit, you can easily compute the gradients of circuit properties including eigenfrequencies, decoherence times, and matrix elements with respect to the values of the circuit's elements. This notebook shows you how to do that.

Make sure you've acquainted yourself with the basic operation of SQcircuit first (find a quick tutorial [here](https://docs.sqcircuit.org/quick_tutorial.html)).

SQcircuit performs almost all the gradient computation automatically. There are only two things you need to do differently when you want to compute gradients.

### 1. Change the numerical engine of SQcircuit to PyTorch
Normally, SQcircuit uses NumPy and QuTiP internally. However, to compute gradients we need to switch to using PyTorch (a machine-learning library). This is done using SQcircuit's `set_engine` command.

In [1]:
import SQcircuit as sq
sq.set_engine('PyTorch')

### 2. Indicate the elements to compute gradients with respect to

When creating an element, you can decide if you need to compute gradients with respect to it. To do so, pass in the `requires_grad=True` argument. SQcircuit can compute gradients with respect to
- Capacitors
- Inductors
- Junctions
- Loops (i.e. external flux)

In [33]:
# Let's create a flux-tunable transmon
# We'll enable gradient computation for all the elements

loop = sq.Loop(0.5, requires_grad=True)
J1 = sq.Junction(1, 'GHz', loops=[loop], requires_grad=True)
J2 = sq.Junction(2, 'GHz', loops=[loop], requires_grad=True)
C = sq.Capacitor(0.5, 'GHz', requires_grad=True)

circuit = sq.Circuit(
    {(0, 1): [J1, J2, C]}
)

Now, you can compute anything just like when using `SQcircuit` normally! For example, the qubit frequency:

In [34]:
circuit.set_trunc_nums([60])
circuit.diag(10)
qubit_frequency =  circuit.efreqs[1] - circuit.efreqs[0]
print(qubit_frequency)

tensor(2.1861, dtype=torch.float64, grad_fn=<SubBackward0>)


After computing a property like the qubit frequency, we calculate the gradient of it with respect to the element values by calling `.backward()`.

In [35]:
qubit_frequency.backward()

The gradients can now be accessed via the `.grad` attribute of the elements:

In [36]:
print('Gradients of qubit frequency with respect to')
print('\t capacitor:', C.grad)
print('\t junction 1:', J1.grad)
print('\t junction 2:', J2.grad)
print('\t external flux:', loop.grad)

Gradients of qubit frequency with respect to
	 capacitor: tensor(-4.7772e+13, dtype=torch.float64)
	 junction 1: tensor(-5.3377e-11, dtype=torch.float64)
	 junction 2: tensor(5.3377e-11, dtype=torch.float64)
	 external flux: tensor(5.8639e-08)


And that's all there is to it! This works for just just about anything else that `SQcircuit` can compute.

From decoherence times…

In [37]:
circuit.zero_grad() # see Gotchas below

t1 = 1 / (
    circuit.dec_rate('capacitive', (0, 1))
    + circuit.dec_rate('inductive', (0, 1))
    + circuit.dec_rate('quasiparticle', (0, 1))
)

print('T1 time:', t1)

t1.backward()

print('Gradients of T1 time with respect to')
print('\t capacitor:', C.grad)
print('\t junction 1:', J1.grad)
print('\t junction 2:', J2.grad)
print('\t external flux:', loop.grad)

T1 time: tensor(0.0004, dtype=torch.float64, grad_fn=<MulBackward0>)
Gradients of T1 time with respect to
	 capacitor: tensor(4.4872e+08, dtype=torch.float64)
	 junction 1: tensor(1.1094e-13, dtype=torch.float64)
	 junction 2: tensor(-1.1094e-13, dtype=torch.float64)
	 external flux: tensor(-1.2188e-10)


…to matrix elements like
$$ g_{10} = \frac{1}{C}|\bra{1}{\hat{Q}}\ket{0}| $$

In [38]:
circuit.zero_grad()

import torch
g_10 = (1/C.get_value()) * torch.abs(
    circuit.evecs[1].conj()
    @ circuit.charge_op(1)
    @ circuit.evecs[0]
)

print('g_{10}:', g_10)

g_10.backward()

print('Gradients of g_{10} time with respect to')
print('\t capacitor:', C.grad)
print('\t junction 1:', J1.grad)
print('\t junction 2:', J2.grad)
print('\t external flux:', loop.grad)

g_{10}: tensor(2.4835e+11, dtype=torch.float64, grad_fn=<MulBackward0>)
Gradients of g_{10} time with respect to
	 capacitor: tensor(-1.4721e+24, dtype=torch.float64)
	 junction 1: tensor(-30.4499, dtype=torch.float64)
	 junction 2: tensor(30.4499, dtype=torch.float64)
	 external flux: tensor(33451.8086)


One of the main applications of these gradients is to use during optimization. See the other [TODO] example notebook for a tutorial on how to do that from scratch, or use the ready-built [Qubit-Discovery](https://github.com/stanfordLINQS/Qubit-Discovery) module!

## Gotchas

Since SQcircuit uses PyTorch as a backend to compute gradients, there are a few things to be aware of if you haven't used PyTorch before.

### 1. Gradients add unless cleared

Suppose you calculate the gradient of the qubit frequency

```
qubit_frequency = circuit.efreqs[1] - circuit.efreqs[0]
qubit_frequency.backward()
```

and then calculate the gradient of the $T_1$ time due to capacitive loss
```
T1 = circuit.dec_rate('capacitive', (0, 1))
T1.backward()
```

The gradient of the elements will not be the gradient of just $T_1$, but instead the gradient of $T_1$ **plus** the gradient of $\omega_{10}$. This is because every time you call `.backward()`, PyTorch adds the gradient on. 

To fix this, you must reset the gradients to zero. To do this, call the `.zero_grad()` method of the circuit. 


### 2. You can't call `.backward()` twice

Suppose you compute the qubit frequency and compute the gradients by calling `.backward()` on it.

```
qubit_frequency = cr.efreqs[1] - cr.efreqs[0]
qubit_frequency.backward()
```

If you need to recompute the gradient of `qubit_frequency` (for example, if you've zeroed the gradients in-between), calling `.backward()` on `qubit_frequency` again won't work. PyTorch automatically deletes some internal data after calling `.backward()` to save memory.

Instead, you need to recompute the value, and then call
```
qubit_frequency = cr.efreqs[1] - cr.efreqs[0]
qubit_frequency.backward()
```

If you know in advance you're going to need to recompute the gradient later, you can let PyTorch know by passing in `retain_graph=True` to `.backward()`.
```
qubit_frequency.backward(retain_graph=True)
```


### Takeaways

* Always call `circuit.zero_grad()` between calculating gradients of different functions.
* Don't call `.backward()` twice on the same value; to recompute a gradient you must recompute the value.